<a href="https://colab.research.google.com/github/habiib1999/Speech-Emotion-Recognition/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# READ .WAV FILES AND LISTEN TO THEM

In [50]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio
import os
import re

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Audio
# from entropy import spectral_entropy
from keras import layers
from keras import models
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import itertools


# Class : READER

In [51]:
#READER CLASS
class Reader:
  def __init__(self, ravdess_path, crema_path, savee_path, tess_path):
    self.ravdess_path = ravdess_path
    self.crema_path = crema_path
    self.savee_path = savee_path
    self.tess_path = tess_path

  def readRavdess(self):
    ravdess_directory_list = os.listdir(self.ravdess_path)

    emotion_df = []

    for dir in ravdess_directory_list:
      actor = os.listdir(os.path.join(ravdess_path, dir))
      for wav in actor:
          info = wav.partition(".wav")[0].split("-")
          emotion = int(info[2])
          emotion_df.append((emotion, os.path.join(ravdess_path, dir, wav)))


    Ravdess_df = pd.DataFrame.from_dict(emotion_df)
    Ravdess_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)
    Ravdess_df.Emotion.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
    #print(Ravdess_df.head())
    return Ravdess_df

  def readCrema(self):
    emotion_df = []

    for wav in os.listdir(crema_path):
      info = wav.partition(".wav")[0].split("_")
      if info[2] == 'SAD':
        emotion_df.append(("sad", crema_path + "/" + wav))
      elif info[2] == 'ANG':
        emotion_df.append(("angry", crema_path + "/" + wav))
      elif info[2] == 'DIS':
        emotion_df.append(("disgust", crema_path + "/" + wav))
      elif info[2] == 'FEA':
        emotion_df.append(("fear", crema_path + "/" + wav))
      elif info[2] == 'HAP':
        emotion_df.append(("happy", crema_path + "/" + wav))
      elif info[2] == 'NEU':
        emotion_df.append(("neutral", crema_path + "/" + wav))
      else:
        emotion_df.append(("unknown", crema_path + "/" + wav))


    Crema_df = pd.DataFrame.from_dict(emotion_df)
    Crema_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)

    #print(Crema_df.head())
    return Crema_df

  def readSavee(self):
    savee_directiory_list = os.listdir(savee_path)
    emotion_df = []
    for wav in savee_directiory_list:
      info = wav.partition(".wav")[0].split("_")[1].replace(r"[0-9]", "")
      emotion = re.split(r"[0-9]", info)[0]
      if emotion=='a':
        emotion_df.append(("angry", savee_path + "/" + wav))
      elif emotion=='d':
        emotion_df.append(("disgust", savee_path + "/" + wav))
      elif emotion=='f':
        emotion_df.append(("fear", savee_path + "/" + wav))
      elif emotion=='h':
        emotion_df.append(("happy", savee_path + "/" + wav))
      elif emotion=='n':
        emotion_df.append(("neutral", savee_path + "/" + wav))
      elif emotion=='sa':
        emotion_df.append(("sad", savee_path + "/" + wav))
      else:
        emotion_df.append(("surprise", savee_path + "/" + wav))

    Savee_df = pd.DataFrame.from_dict(emotion_df)
    Savee_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)
    #print(Savee_df.head())
    return Savee_df

  def readTess(self):  
    tess_directory_list = os.listdir(tess_path)

    emotion_df = []

    for dir in tess_directory_list:
      for wav in os.listdir(os.path.join(tess_path, dir)):
        info = wav.partition(".wav")[0].split("_")
        emo = info[2]
        if emo == "ps":
            emotion_df.append(("surprise", os.path.join(tess_path, dir, wav)))
        else:
            emotion_df.append((emo, os.path.join(tess_path, dir, wav)))


    Tess_df = pd.DataFrame.from_dict(emotion_df)
    Tess_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)

    #print(Tess_df.head())
    return Tess_df

  def read(self):
    ravdess_dataset = self.readRavdess()
    crema_dataset = self.readCrema()
    savee_dataset = self.readSavee()
    tess_dataset = self.readTess()
    return ravdess_dataset, crema_dataset, savee_dataset, tess_dataset
    
  def concatenate(self,ravdess_dataset,crema_dataset,savee_dataset,tess_dataset):
    concat_dataset = pd.concat([ravdess_dataset,crema_dataset,savee_dataset,tess_dataset], axis=0)
    print(concat_dataset.shape)
    return concat_dataset

# Listen or plot Audios

In [52]:
def listenToAudio(path):
  frequency_sampling, audio_signal = wavfile.read(path)
  print('\nSignal shape:', audio_signal.shape)
  print('Signal Datatype:', audio_signal.dtype)
  print('Signal duration:', round(audio_signal.shape[0] / 
  float(frequency_sampling), 2), 'seconds')
  audio_signal = audio_signal / np.power(2, 15)
  audio_signal = audio_signal [:100]
  time_axis = 1000 * np.arange(0, len(audio_signal), 1) / float(frequency_sampling)
  plt.plot(time_axis, audio_signal, color='blue')
  plt.xlabel('Time (milliseconds)')
  plt.ylabel('Amplitude')
  plt.title('Input audio signal')
  plt.show()
  Audio(path)

In [53]:
ravdess_path = "/content/drive/MyDrive/emotionDataset/Ravdess/audio_speech_actors_01-24"
crema_path = "/content/drive/MyDrive/emotionDataset/Crema"
savee_path = "/content/drive/MyDrive/emotionDataset/Savee"
tess_path = "/content/drive/MyDrive/emotionDataset/Tess"

reader = Reader(ravdess_path, crema_path, savee_path, tess_path)
ravdess_dataset,crema_dataset,savee_dataset,tess_dataset = reader.read()
concat_dataset = reader.concatenate(ravdess_dataset,crema_dataset,savee_dataset,tess_dataset)
from sklearn.utils import shuffle
concat_dataset = shuffle(concat_dataset)
print(concat_dataset.head(10))


(12162, 2)
       Emotion                                               Path
5965   disgust  /content/drive/MyDrive/emotionDataset/Crema/10...
3121     happy  /content/drive/MyDrive/emotionDataset/Crema/10...
6330     happy  /content/drive/MyDrive/emotionDataset/Crema/10...
5270   neutral  /content/drive/MyDrive/emotionDataset/Crema/10...
6779     happy  /content/drive/MyDrive/emotionDataset/Crema/10...
4763   disgust  /content/drive/MyDrive/emotionDataset/Crema/10...
3558       sad  /content/drive/MyDrive/emotionDataset/Crema/10...
593   surprise  /content/drive/MyDrive/emotionDataset/Tess/OAF...
1528     happy  /content/drive/MyDrive/emotionDataset/Tess/YAF...
2266     happy  /content/drive/MyDrive/emotionDataset/Crema/10...


# Create feature space

In [54]:
#reader = Reader()
def energy(data, frame_length=2048, hop_length=512):
    en = np.array([np.sum(np.power(np.abs(data[hop:hop+frame_length]), 2)) for hop in range(0, data.shape[0], hop_length)])
    return en / frame_length
def mel_spc(data, sr, frame_length=2048, hop_length=512, flatten: bool = False):
    mel = librosa.feature.melspectrogram(y=data, sr=sr)
    return np.squeeze(mel.T) if not flatten else np.ravel(mel.T)

In [55]:
def extract_features(data, sr, frame_length=2048, hop_length=512):
    result = np.array([])

    return np.mean(energy(data, frame_length, hop_length),axis=0),mel_spc(data, sr, frame_length, hop_length)


def get_features(path, duration=2.5, offset=0.6):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=duration, offset=offset)

    energy_feature,mel_spec_feature = extract_features(data, sample_rate)

    return np.array(energy_feature) , np.array(mel_spec_feature)

In [56]:
path = np.array(concat_dataset["Path"])[657]
data, sampling_rate = librosa.load(path, duration=2.5, offset=0.6)
print(len(data))
print(data)
print(sampling_rate)
enrg = energy(data)
mel_spec = mel_spc(data, sampling_rate)
print("Energy shape: ", enrg.shape)
print("Energy: ", enrg)
print("MelSpectrogram shape: ", mel_spec.shape)
print("MelSpectrogram : ", mel_spec)

34092
[-0.07679766 -0.0756614  -0.05751546 ...  0.00024126  0.00014337
  0.        ]
22050
Energy shape:  (67,)
Energy:  [4.8163551e-04 3.0414090e-05 2.3859879e-05 7.7102071e-04 1.3334990e-03
 1.6183523e-03 1.7789254e-03 1.1335369e-03 6.4440852e-04 4.5603770e-04
 3.7313387e-04 3.2921013e-04 3.2670380e-04 3.4616533e-04 5.4895977e-04
 1.2826503e-03 2.3805392e-03 3.4723037e-03 3.9064512e-03 4.2178491e-03
 4.7476948e-03 4.9375570e-03 5.3153103e-03 5.3374292e-03 5.2532591e-03
 4.6475576e-03 4.2079724e-03 3.2272479e-03 1.6557055e-03 9.0297230e-04
 2.5838485e-04 1.0806081e-04 6.2823441e-05 2.1896602e-05 2.4189177e-07
 1.1852012e-07 9.7806556e-08 6.5660997e-06 1.1994374e-04 4.1365516e-04
 6.6685735e-04 7.3722657e-04 6.7886867e-04 5.3633685e-04 5.6321366e-04
 1.1981376e-03 2.4483732e-03 3.9005042e-03 5.2773296e-03 6.1781430e-03
 5.9975903e-03 4.6594786e-03 3.0077444e-03 1.3953907e-03 2.7087214e-04
 5.7553611e-06 6.1523588e-07 1.2171058e-05 2.5469450e-05 3.4303637e-05
 3.8254479e-05 2.7815133e-0

In [57]:
extracted_data_energy, extracted_data_mel_spec = [], []
extracted_data_energy_labels, extracted_data_mel_spec_labels = [], []
print("Feature processing...")
for path, emotion, ind in zip(concat_dataset.Path, concat_dataset.Emotion, range(concat_dataset.Path.shape[0])):
    energy_feature,mel_spec_feature = get_features(path)
    if ind % 100 == 0:
        print(f"{ind} samples has been processed...")
    extracted_data_energy.append(energy_feature)      
    extracted_data_energy_labels.append(emotion)
    extracted_data_mel_spec.append(mel_spec_feature)      
    extracted_data_mel_spec_labels.append(emotion)

print("Done.")

Feature processing...
0 samples has been processed...
100 samples has been processed...
200 samples has been processed...
300 samples has been processed...
400 samples has been processed...
500 samples has been processed...
600 samples has been processed...
700 samples has been processed...
800 samples has been processed...
900 samples has been processed...
1000 samples has been processed...
1100 samples has been processed...
1200 samples has been processed...
1300 samples has been processed...
1400 samples has been processed...
1500 samples has been processed...
1600 samples has been processed...
1700 samples has been processed...
1800 samples has been processed...
1900 samples has been processed...
2000 samples has been processed...
2100 samples has been processed...
2200 samples has been processed...
2300 samples has been processed...
2400 samples has been processed...
2500 samples has been processed...
2600 samples has been processed...
2700 samples has been processed...
2800 sampl

In [58]:
features_path = "/content/drive/MyDrive/emotionDataset/energy_features.csv"
extracted_df = pd.DataFrame(extracted_data_energy)
extracted_df["labels"] = extracted_data_energy_labels
extracted_df.to_csv(features_path, index=False)
extracted_df.head()

,0,labels
0,0.001820,disgust
1,0.000259,happy
2,0.000807,happy
3,0.001235,neutral
4,0.001565,happy


In [59]:
features_path = "/content/drive/MyDrive/emotionDataset/mel_spec_features.csv"
extracted_df = pd.DataFrame(extracted_data_mel_spec)
extracted_df["labels"] = extracted_data_mel_spec_labels
extracted_df.to_csv(features_path, index=False)
extracted_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


,0,labels
0,"[[0.037761945, 0.16695678, 0.13421017, 0.04242...",disgust
1,"[[0.056401253, 0.06830928, 0.062207915, 0.0446...",happy
2,"[[0.032273035, 0.032312907, 0.049332775, 0.030...",happy
3,"[[0.009226999, 0.030860092, 0.014787596, 0.006...",neutral
4,"[[0.09210995, 0.04682401, 0.02212438, 0.300461...",happy


In [91]:
features_path = "/content/drive/MyDrive/emotionDataset/energy_features.csv"
extracted_df = pd.read_csv(features_path)
extracted_df.head()
print(len(extracted_df.loc[4][0]))

563
